In [1]:
import pandas as pd
import numpy as np

candidate_data = pd.read_csv('/home/keonwoo/anaconda3/envs/bgmRS/data/Audio library - 500.csv',encoding='utf-8')
candidate_data.drop(['Unnamed: 0','순서'],axis=1,inplace=True)
candidate_data = candidate_data.fillna(0)

mood_list = np.unique(candidate_data['메인무드'])
mood_list_ko = ['쿨한', '신나는', '우스운', '멋진', '평화로운', '로맨틱한', '무서운, 소름 끼치는, 미스터리한', '감동적인', '희망적인']
mood_dict = dict(zip(mood_list, mood_list_ko))

def change_label(x):
    return mood_dict[x]
    
candidate_data['메인무드'] = candidate_data['메인무드'].apply(lambda x: change_label(x))


from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import torch

# device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device = torch.device("cpu")
ckpt_path = "monologg/kobigbird-bert-base" # 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'
finetuned_model_ckpt = "bigbird_0707"

sbert_tokenizer = AutoTokenizer.from_pretrained(ckpt_path)
model = AutoModelForSequenceClassification.from_pretrained(f"/home/keonwoo/anaconda3/envs/bgmRS/ckpt/{finetuned_model_ckpt}", num_labels=9)
# model = AutoModelForSequenceClassification.from_pretrained("/home/keonwoo/anaconda3/envs/bgmRS/ckpt/sbert_0706", num_labels=9)
model.to(device)

/home/keonwoo/anaconda3/envs/bgmRS/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BigBirdForSequenceClassification(
  (bert): BigBirdModel(
    (embeddings): BigBirdEmbeddings(
      (word_embeddings): Embedding(32500, 768, padding_idx=0)
      (position_embeddings): Embedding(4096, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BigBirdEncoder(
      (layer): ModuleList(
        (0): BigBirdLayer(
          (attention): BigBirdAttention(
            (self): BigBirdBlockSparseAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): BigBirdSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [2]:
def recommend_mood(text):
    label_dict = {
        '희망적인' : 0,  
        '감동적인' : 1, 
        '무서운, 소름 끼치는, 미스터리한' : 2,
        '로맨틱한' : 3, 
        '평화로운' : 4,
        '멋진' : 5, 
        '우스운' : 6,
        '신나는' : 7, 
        '쿨한' : 8 
    }

    reversed_dict = dict(map(reversed, label_dict.items()))

    encoded_input = sbert_tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    model.eval()
    with torch.no_grad():
        model_output = model(**encoded_input)

    output = torch.argmax(model_output[0][0])

    pred_mood = reversed_dict[int(output.numpy())]

    recommended_song_df = candidate_data[candidate_data['메인무드'] == pred_mood]

    print("적절한 테마를 고르시오: ", np.unique(recommended_song_df['테마']))

    return recommended_song_df

def recommend_song(recommended_song_df, tema, num_song=3):
    # tema = 'News / Current Affair'
    # num_song = 3

    final_recommended = recommended_song_df[recommended_song_df['테마'] == tema]

    if len(final_recommended) < num_song:
        print(final_recommended['곡제목(임시)'])
        RS = final_recommended['곡제목(임시)']
    else:
        print(final_recommended.sample(n=num_song)['곡제목(임시)'])
        RS = final_recommended.sample(n=num_song)['곡제목(임시)']

    return RS 

In [4]:
# 정확

text = """강호순과 조두순의 공통점은? 둘 다 동물을 학대 했다는 점이다. 2006년부터 3년간 부녀자 등 10명을 살해한 강호순은 첫 범행에 앞서 자신이 운영하는 개 사육장에서 개들을 잔인하게 죽였다고 알려졌다. 강호순은 프로파일러 면담 중에도 "개를 많이 죽이다 보니 사람 죽이는 것도 아무렇지 않고 살인 욕구를 자제할 수 없었다"고 했다.

아동성범죄자 조두순은 반려견의 눈을 찔러 죽였다. 조두순은 조사 중에도 검사가 '술에 취해 자신도 모르게 이상한 행동을 한 적이 있냐'고 묻자 "강아지에게 병을 집어던져 죽인 적이 두 번 있었다" "그중 한 마리의 눈을 빗자루 몽둥이로 찔러 죽였다"라고 했다.

전문가들은 동물을 향한 폭력성이 사람에게 향할 수 있다고 경고한다. 인명피해가 없다고 동물학대범를 가벼이 여겨서는 안되는 이유다."""
text = text.replace('\n','')

df = recommend_mood(text)

print('\n')

songs = recommend_song(df, "News / Current Affair").values

적절한 테마를 고르시오:  [' Video Game' 'Advertising' 'Comedy' 'Film / Drama / Dacumentary'
 'History' 'Lifestyle / Leisure' 'News / Current Affair' 'SF'
 'Time Lapse & Slomo' 'Video Game']


41     Behind_story
118      Deep_touch
100           Count
Name: 곡제목(임시), dtype: object


In [ ]:
import os

song_path = "/home/keonwoo/anaconda3/envs/bgmRS/data/500_wav_files"

dir_list = os.listdir(song_path)

for name in dir_list:
    src = os.path.join(song_path, name)
    only_name = ("_".join(name.split('_')[1:])).lower()
    dst = song_path + "/" + only_name
    os.rename(src, dst)

In [5]:
song_path = "/home/keonwoo/anaconda3/envs/bgmRS/data/500_wav_files"

dir_list = os.listdir(song_path)

for rs_song in songs:
    song_idx = dir_list.index(f"{rs_song}".lower() + '.mp3')

    song_path_find = os.path.join(song_path, dir_list[song_idx])
    print(song_path_find)

/home/keonwoo/anaconda3/envs/bgmRS/data/500_wav_files/count.mp3
/home/keonwoo/anaconda3/envs/bgmRS/data/500_wav_files/architecture.mp3
/home/keonwoo/anaconda3/envs/bgmRS/data/500_wav_files/behind_story.mp3


In [11]:
text = """
포항제철소가 용기있는 행동을 한 지역 거주 의인을 초청해 감사의 마음을 전하는 뜻깊은 행사를 가졌다.

포항제철소는 지난 26일 오후 7시 포항스틸야드에서 펼쳐진 포항스틸러스와 김천상무의 경기 킥오프 전, 포항 지역에 거주하는 의인(義人) 과 가족·동료를 초청해 시축 행사를 진행하고 경기를 함께 관전하며 응원했다.

이번 행사에는 ‘포스코히어로즈’로 선정된 이수형(포스코휴먼스 대리) 씨, 최다래·박진수(대학생) 씨와 포항남부경찰서장 표창을 수여한 이성욱(포롤텍 대리) 씨 등 4명의 ‘시민영웅’이 초청돼 자리를 빛냈다.

포스코청암재단이 운영하는 ‘포스코히어로즈’ 펠로십은 국가와 사회정의를 위해 살신성인의 자세로 자신을 희생한 의인이나 의인의 자녀가 안정적으로 학업을 계속할 수 있도록 지원하는 사업이다.

사회적 귀감이 되는 사례를 적극적으로 발굴해 의로운 행동에 대한 사회적 관심을 높이고, 우리 사회 구성원들이 어려운 상황에서도 서로 도울 수 있는 정의로운 사회를 만들기 위해 2019년 제정됐으며, 현재까지 총 57명의 포스코히어로즈가 선정됐다.

이수형 씨는 지난 2019년 화재로 불타고 있는 주택에 뛰어들어 일가족 4명의 생명을 구했고, 최다래·박진수 씨는 지난 1월 포항시 북구 원룸에서 발생한 화재 현장에서 불길에 뛰어들어 잠든 주민 7명을 깨워 대피시켜 인명 피해를 막았다.
"""
text = text.replace('\n', '')

df = recommend_mood(text)

songs = recommend_song(df, "News / Current Affair")

In [12]:
text = """
전주연탄은행(대표 윤국춘)은 9일 ‘참! 좋은 사랑의 밥차’로 도시락 200개를 준비해 전주지역 독거노인 및 소외계층가정에 전달하였다.

전주연탄은행은 2015년부터 IBK기업은행으로부터 밥차를 기증받은 이래, 매년 한국자원봉사센터협회를 통해 운영예산을 후원받아 ‘참! 좋은 사랑의 밥차’ 무료급식을 7년째 운영하고 있으며, 22년도 밥차봉사는 지난 4월부터 시작하여 매주 도시락 200개를 준비, 전북지역에 거주하는 독거노인 및 소외계층(저소득·장애인·차상위·한부모) 가정이 따뜻한 한끼를 드실 수 있도록 전달해오고 있다.

윤국춘 전주연탄은행 대표는 “사랑의 밥차를 운영할 수 있도록 후원해주시는 기업은행과 한국자원봉사센터협회에 감사드린다“며 “전북지역에 거주하는 독거노인 및 소외계층 가정이 어려운 상황에서도 건강을 유지할 수 있도록 따뜻한 식사 한끼를 대접하고자 노력하고 있다”고 말했다.

전주연탄은행은 ‘참! 좋은 사랑의 밥차’ 무료배식 나눔봉사를 통해 도시락 및 밑반찬, 삼계탕 등을 매주 1~2회씩, 10월말까지 운영할 예정이다.

"""
text = text.replace('\n', '')

df = recommend_mood(text)

songs = recommend_song(df, "News / Current Affair")

In [18]:
text = """
약속해 주겠니 이담에 우리 자라서 너와 내가 어른 되면

꼭 넌 나와 결혼 할꺼라는 말

 

얼마나 더 간거니 네가 좋아 했던 곳 많이 널 아껴 왔던 내 품 안에서,

아마 살아 날 떠났다면 널 잊는 것도 나 사는 것도,

익숙해 질 날이 올텐데 네가 남겨 둔 눈물로 젖은 편지엔,

잘 살아 달라고 행복해 달라고 너의 반지도 또누군갈 위해서,

주라 했지만 못다 했던 그 말 이제 전하려 해 기다릴 수 있겠니.

 

조금 더디 걸려도 너의 손에 나의 약속을 되돌릴 그날 까지,

사랑할 수 있겠니 너의 끝이 나이듯 나의 마지막 사랑

이 세상엔 네가 될 수 있게.

 

세월이 흘러 내 모습이 변해 갈 그 날에도,

내 기억 속엔 언제나 그대로야 다음 세상엔 제발

늦게 만났으면 해 너와 내 운명의 시간이 정해진 것이라면,

우리 서로 같은 날 함께 마칠 수 있게 다시는 먼 그 곳에,

널 혼자 보내는 일 없게 꼭 이것 만은 약속해 줄께.
"""
text = text.replace('\n','')

df = recommend_mood(text)

songs = recommend_song(df, "Time Lapse & Slomo")